## Links

https://python.langchain.com/docs/modules/agents/

## Required Imports

In [18]:
import os
from langchain.tools.render import format_tool_to_openai_function
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chat_models import ChatOpenAI
from langchain.agents import tool
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents import AgentExecutor
from langchain.agents import initialize_agent, AgentType
from langchain.schema.messages import AIMessage, HumanMessage, SystemMessage
from langchain.globals import set_verbose, set_debug

debug=False
set_verbose(debug)
set_debug(debug)

OpenAPIKey = os.environ.get("OPENAI_API_KEY")


## Simple Tool Test

This is excluding the runtime, hence it will resolve the function call and the arguments but not actually call the function (called tool in langchain).

In [21]:
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

@tool
def get_word_length(word: str) -> int:
    """Returns the length of a word."""
    return len(word)


tools = [get_word_length]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but bad at calculating lengths of words.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm_with_tools = model.bind(functions=[format_tool_to_openai_function(t) for t in tools])

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent.invoke({"input": "how many letters in the word cookie worm?", "intermediate_steps": []})

AgentActionMessageLog(tool='get_word_length', tool_input={'word': 'cookie worm'}, log="\nInvoking: `get_word_length` with `{'word': 'cookie worm'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "word": "cookie worm"\n}', 'name': 'get_word_length'}})])

## Define the Runtime

This will drive the completion of the task.

In [11]:
from langchain.schema.agent import AgentFinish

user_input = "how many letters in the word cookie worm?"
intermediate_steps = []
myTools = {"get_word_length": get_word_length}

while True:
    output = agent.invoke(
        {
            "input": user_input,
            "intermediate_steps": intermediate_steps,
        }
    )
    if isinstance(output, AgentFinish):
        final_result = output.return_values["output"]
        break
    else:
        print(f"TOOL NAME: {output.tool}")
        print(f"TOOL INPUT: {output.tool_input}")
        tool = myTools[output.tool]
        observation = tool.run(output.tool_input)
        intermediate_steps.append((output, observation))
print(final_result)

TOOL NAME: get_word_length
TOOL INPUT: {'word': 'cookie worm'}
There are 11 letters in the word "cookie worm".


## Use a Built-in Executor: AgentExecutor

In [12]:
chain = AgentExecutor(agent=agent, tools=tools, verbose=debug)

chain.invoke({"input": "how many letters in the word cookie worm?"})

{'input': 'how many letters in the word cookie worm?',
 'output': 'There are 11 letters in the word "cookie worm".'}

## Add Memory to the Agent

In [13]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are very powerful assistant, but bad at calculating lengths of words."),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

chat_history = []

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
        "chat_history": lambda x: x["chat_history"],
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

chain = AgentExecutor(agent=agent, tools=tools, verbose=debug)

# First call
input = "how many letters in the word cookie worm?"
result = chain.invoke({"input": input, "chat_history": chat_history})

# Add messages to chat history
chat_history.extend(
    [
        HumanMessage(content=input),
        AIMessage(content=result["output"]),
    ]
)

# Second call
chain.invoke({"input": "is that a real word?", "chat_history": chat_history})

{'input': 'is that a real word?',
 'chat_history': [HumanMessage(content='how many letters in the word cookie worm?'),
  AIMessage(content='There are 11 letters in the word "cookie worm".')],
 'output': '"Cookie worm" is not a real word. It is a combination of two separate words, "cookie" and "worm".'}

## Get all Built-in Tools

In [14]:
from langchain.agents import get_all_tool_names
get_all_tool_names()


['python_repl',
 'requests',
 'requests_get',
 'requests_post',
 'requests_patch',
 'requests_put',
 'requests_delete',
 'terminal',
 'sleep',
 'wolfram-alpha',
 'google-search',
 'google-search-results-json',
 'searx-search-results-json',
 'bing-search',
 'metaphor-search',
 'ddg-search',
 'google-serper',
 'google-scholar',
 'google-serper-results-json',
 'searchapi',
 'searchapi-results-json',
 'serpapi',
 'dalle-image-generator',
 'twilio',
 'searx-search',
 'wikipedia',
 'arxiv',
 'golden-query',
 'pubmed',
 'human',
 'awslambda',
 'sceneXplain',
 'graphql',
 'openweathermap-api',
 'dataforseo-api-search',
 'dataforseo-api-search-json',
 'eleven_labs_text2speech',
 'google_cloud_texttospeech',
 'news-api',
 'tmdb-api',
 'podcast-api',
 'memorize',
 'llm-math',
 'open-meteo-api']

## My First Custom Tool (SMHI)

https://blog.langchain.dev/structured-tools/

In [6]:
import requests
import aiohttp
from requests.exceptions import HTTPError
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field
from typing import Optional, Type


class ForecastInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location")
    longitude: float = Field(..., description="Longitude of the location")

class ForecastOutput(BaseModel):
    temperature: Optional[float] = Field(None, description="Temperature in degrees Celsius")
    wind_speed: Optional[float] = Field(None, description="Wind speed in meters per second")
    precipitation: Optional[float] = Field(None, description="Precipitation in millimeters")

class ForecastTool(StructuredTool):
    name: str = "GetWeatherForecast"
    description: str = "Useful when you need to answer a question about weather in a specific location."
    args_schema: Type[BaseModel] = ForecastInput
        # SMHI API endpoint for weather forecast
    smhi_url = "https://opendata-download-metfcst.smhi.se/api/category/pmp3g/version/2/geotype/point/lon/{input.longitude}/lat/{input.latitude}/data.json"


    def _run(self, latitude: float, longitude: float) -> ForecastOutput:
        print(f"(sync) Retrieving weather forecast for lat: {latitude}, lon: {longitude}")

        url = self.smhi_url.format(input=ForecastInput(latitude=latitude, longitude=longitude))
        response = requests.get(url)

        if response.status_code == 200:
            return self.extract_weather_info(response=response)
        else:
            raise HTTPError(f'Unexpected status code: {response.status_code}')
        
    async def _arun(self, latitude: float, longitude: float) -> ForecastOutput:
        print(f"(async) Retrieving weather forecast for lat: {latitude}, lon: {longitude}")

        url = self.smhi_url.format(input=ForecastInput(latitude=latitude, longitude=longitude))
        async with aiohttp.ClientSession() as session:
            async with session.get(url) as response:
                if response.status == 200:
                    return self.extract_weather_info(response=response)
                else:
                    raise HTTPError(f'Unexpected status code: {response.status_code}')

    def extract_weather_info(self, response: requests.Response) -> ForecastOutput:
        forecast=response.json()
        
        if 'timeSeries' in forecast and len(forecast['timeSeries']) > 0:
            # The first element in the time series is usually the current weather forecast
            current_forecast = forecast['timeSeries'][0]

            weather_info: ForecastOutput = {
                'temperature': None,
                'wind_speed': None,
                'precipitation': None
            }

            for parameter in current_forecast['parameters']:
                if parameter['name'] == 't':  # Temperature
                    weather_info['temperature'] = parameter['values'][0]
                elif parameter['name'] == 'ws':  # Wind speed
                    weather_info['wind_speed'] = parameter['values'][0]
                elif parameter['name'] == 'pmean':  # Mean precipitation
                    weather_info['precipitation'] = parameter['values'][0]

            return weather_info
        else:
            raise KeyError("Error: Could not parse the weather forecast.")


# Replace with the coordinates of your city in Sweden
latitude = "64.7514"  # Example: Skellefteå
longitude = "20.9579"

forecast_tool = ForecastTool()
forecast_tool._run(latitude, longitude)

(sync) Retrieving weather forecast for lat: 64.7514, lon: 20.9579


{'temperature': -12.9, 'wind_speed': 5.2, 'precipitation': 0.1}

## Use SMHI Tool in Agent

https://nanonets.com/blog/langchain/

https://api.python.langchain.com/en/latest/experimental_api_reference.html

In [7]:
prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="You are very powerful assistant, but you need to use a tool get weather info."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm.bind(stop=["Observation"]) # Stops on Observation
    | OpenAIFunctionsAgentOutputParser()
)

chain = initialize_agent(
  agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
  llm=model,
  tools=[ForecastTool()], verbose=debug
)

result = chain.invoke({
  "input": "how cold is it in Skellefteå?",
  "agent_scratchpad": [],
  })

print(result)

(sync) Retrieving weather forecast for lat: 64.75, lon: 20.95
{'input': 'how cold is it in Skellefteå?', 'agent_scratchpad': [], 'output': 'The current temperature in Skellefteå is -12.9 degrees Celsius.'}
